In [1]:
ls = sc.textFile('data/coupon150720.csv')

In [5]:
ls.take(5)
#ls.first()

[u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 u'79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 u'79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0',
 u'79065668570385,1,DEL,DXB,9W,9W,160.63,USD,2,S,S,0546,150804,OK,INA0',
 u'79065668737021,1,AUH,IXE,9W,9W,152.46,USD,1,V,V,0501,150803,OK,INA0']

In [4]:
ls.count()

1232662

In [25]:
# contamos el número de pasajeros que hayan volado con iberia (IB),columna [5]
ls.filter(lambda x: x.split(",")[5] == 'IB').count()

26158

In [26]:
# definimos una función que nos devuelve la aerolínea y el dinero. La entrada es una línea del fichero
def getAerolineaDinero(l):
    elems = l.split(",")
    aerolinea = elems[5]
    dinero = float(elems[6])
    return (aerolinea,dinero)

In [27]:
getAerolineaDinero(ls.first())

(u'9W', 56.79)

In [28]:
# lo mismo pero utilizando map
dineros = ls.map(getAerolineaDinero)

In [29]:
dineros.take(3)

[(u'9W', 56.79), (u'9W', 84.34), (u'9W', 60.0)]

In [33]:
# filtramos y sumamos para que la aerolinea sea solo Iberia
dinerosIB = dineros.filter(lambda x: x[0]=='IB')

In [34]:
dinerosIB.map(lambda x: x[1]).sum()

2828044.4199999194

### Calculamos las rutas con mayor dinero

In [40]:
ls.first()
#aeropuerto origen, aeropuerto destino y precio. Columnas 2,3 y 6
# creamos una funcion para extraer la ruta y el precio del cupón
def getRutaDinero(l):
    elems = l.split(",")
    origen = elems[2]
    destino = elems[3]
    dinero = float(elems[6])
    ruta = origen + "-" + destino
    return (ruta,dinero)

In [41]:
getRutaDinero(ls.first())

(u'MAA-AUH', 56.79)

In [42]:
# ahora buscamos las rutas top 5 con más dinero
rutas = ls.map(getRutaDinero)

In [43]:
rutas.take(3)

[(u'MAA-AUH', 56.79), (u'AUH-CDG', 84.34), (u'CJB-MAA', 60.0)]

In [46]:
#ordenamos por dinero para ver cuál es la que tiene el dinero mayor
rutas.sortBy(lambda x: x[1],False).take(5)
# estos son los cupones más caros, no las rutas

[(u'SYD-CGK', 6355194.0),
 (u'SYD-CGK', 6355194.0),
 (u'SYD-CGK', 6355194.0),
 (u'CDG-IAD', 637313.52),
 (u'IAD-CDG', 637313.52)]

In [48]:
#quitamos los billetes con un importe superior a 
rutas.filter(lambda x: x[1] < 7500).sortBy(lambda x: x[1],False).take(5)

[(u'BLA-MIA', 7490.33),
 (u'MIA-CCS', 7415.0),
 (u'MIA-CCS', 7415.0),
 (u'CCS-MIA', 7415.0),
 (u'MIA-CCS', 7415.0)]

In [49]:
filtrado = rutas.filter(lambda x: x[1] < 7500)

In [51]:
filtrado.cache()

PythonRDD[60] at RDD at PythonRDD.scala:43

In [52]:
filtrado.groupByKey().take(3)

[(u'SZF-IST', <pyspark.resultiterable.ResultIterable at 0x7effc8f3ef10>),
 (u'DME-RHO', <pyspark.resultiterable.ResultIterable at 0x7effc8f3ea50>),
 (u'KRK-OSL', <pyspark.resultiterable.ResultIterable at 0x7effc8f3e690>)]

In [59]:
maximos = filtrado.groupByKey().mapValues(lambda x: max(x))

In [56]:
maximos.sortBy(lambda x:x[1],False).take(5)

[(u'BLA-MIA', 7490.33),
 (u'MIA-CCS', 7415.0),
 (u'CCS-MIA', 7415.0),
 (u'RUH-LAX', 7401.94),
 (u'DXB-MAA', 7343.27)]

In [67]:
media = filtrado.groupByKey().mapValues(lambda x: sum(x)/len(x))

In [68]:
media.sortBy(lambda x:x[1],False).take(5)

[(u'BLA-MEC', 5164.355),
 (u'MEC-BLA', 5164.355),
 (u'CCS-MIA', 4350.516296296297),
 (u'MIA-CCS', 4186.95),
 (u'NRT-PPT', 3751.5499999999997)]

### vamos a agrupar por aerolinea (la que vende), ruta, booking class

In [69]:
ls.first()
#columnas 2,3,6,8

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [76]:
def getRutaALDineroClase(l):
    elems = l.split(",")
    ruta = elems[2] + "-" + elems[3]
    al = elems[4] #aerolinea que vende el billete
    dinero = float(elems[6])
    clase = elems[9]
    return ((ruta,al,clase),dinero) # la clave son los no numéricos, ruta,clase y al

In [77]:
getRutaALDineroClase(ls.first())

((u'MAA-AUH', u'9W', u'H'), 56.79)

In [78]:
# calculamos el valor límite de cada grupo
# quitamos outliers: media y desviación estándar
import math

In [113]:
def valorLimite(vs): # vs es una lista de valores
    logs = [math.log(v+1) for v in vs]
    #usamos map porque vale tanto para listas como para iterables así no tendremos problemas si al final vs no 
            #es una lista de valores
    media = sum(logs)/len(logs)
    desv = [(x-media)**2 for x in logs]
    varianza = sum(desv)/len(desv)
    desvStd = math.sqrt(varianza)
    return math.exp(media + 2*desvStd)-1

In [114]:
#comprobamos la funcion
valorLimite([12,3,4,1,2,4])

13.060852649090572

In [115]:
rutas = ls.map(getRutaALDineroClase)
rutas.take(5)

[((u'MAA-AUH', u'9W', u'H'), 56.79),
 ((u'AUH-CDG', u'9W', u'H'), 84.34),
 ((u'CJB-MAA', u'9W', u'H'), 60.0),
 ((u'DEL-DXB', u'9W', u'S'), 160.63),
 ((u'AUH-IXE', u'9W', u'V'), 152.46)]

In [116]:
#agrupamos, calculamos el límite para cada grupo y después aplicamos ese límite para cada grupo
##rutas.groupByKey().mapValues(lambda x: ...)

# creamos una función que filtre los outliers para después añadirla en nuestro groupByKey
def filtraCuponesExtremos(vs):
    vl = valorLimite(vs)
    f = [v for v in vs if v < vl]
    return f

In [119]:
grupos = rutas.groupByKey().mapValues(filtraCuponesExtremos)

In [120]:
def mediaGrupo(vs):
    if(len(vs)==0):
        return 0.0
    else:
        return sum(vs)/len(vs)

In [121]:
#grupos.mapValues(lambda vs: sum(vs)/len(vs)).take(5)
grupos.mapValues(mediaGrupo).take(5)

[((u'TRD-BOO', u'SK', u'V'), 42.30263157894736),
 ((u'DPS-TMC', u'GA', u'N'), 60.7925),
 ((u'BEG-CDG', u'JU', u'K'), 0.0),
 ((u'SAV-CLT', u'US', u'N'), 0.0),
 ((u'DUS-AMS', u'KL', u'C'), 0.0)]

In [122]:
# ordenamos 
grupos.mapValues(mediaGrupo).sortBy(lambda x: x[1],False).take(5)
#nos siguen apareciendo cupones de 6 millones, seguramente solo haya 1 cupón con esa PK

[((u'SYD-CGK', u'GA', u'G'), 6355194.0),
 ((u'COO-CDG', u'AF', u'J'), 538599.2),
 ((u'CDG-COO', u'AF', u'J'), 179628.3533333333),
 ((u'DKR-NBO', u'KQ', u'T'), 58253.593333333345),
 ((u'TSN-ICN', u'KE', u'Z'), 38655.45)]

In [126]:
# vamos a quitar los grupos que tengan menos de 10 cupones. Vamos a ver qué tiene grupos:
grupos = rutas.groupByKey()
grupos.take(2)
# = rutas.groupByKey().mapValues(filtraCuponesExtremos)

[((u'TRD-BOO', u'SK', u'V'),
  <pyspark.resultiterable.ResultIterable at 0x7effc105c410>),
 ((u'DPS-TMC', u'GA', u'N'),
  <pyspark.resultiterable.ResultIterable at 0x7effc105c9d0>)]

In [135]:
#nos quedamos con los grupos que tengan más de 10 valores
gruposFiltrados = grupos.filter(lambda x:len(x[1]) > 10)

In [136]:
r = gruposFiltrados.mapValues(filtraCuponesExtremos).mapValues(mediaGrupo)

In [137]:
r.sortBy(lambda x: x[1],False).take(5)

[((u'MIA-CCS', u'S3', u'S'), 7826.25),
 ((u'CCS-MIA', u'S3', u'B'), 5550.0),
 ((u'CCS-MAD', u'V0', u'Y'), 5418.098666666667),
 ((u'RUH-FRA', u'LH', u'F'), 4792.983),
 ((u'LHR-DOH', u'QR', u'F'), 4275.98923076923)]

### cuántas aerolíneas diferentes hay en estos datos

In [138]:
#la función distinct permite reducir el dataset a los valores distintos
ls.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [142]:
ls.map(lambda x: x.split(",")[4]).distinct().count()

359

### comprobamos que todos los precios están en US$

In [143]:
ls.map(lambda x: x.split(",")[7]).distinct().collect()

[u'USD']

In [ ]:
### Guardamos los datos en un directorio en el que hay un fichero por cada partición

In [145]:
ls.map(lambda x: x.split(",")[4]).distinct().saveAsTextFile("results.csv")
#OJO, spark siempre rechaza sobreescribir el directorio de salida